<a href="https://colab.research.google.com/github/vidutyshev/flowers/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Если что-то с зависимостями
!pip install --upgrade torch torchvision sympy

In [ ]:
# Ссылка на архив на Google Диск с датасетом с https://www.kaggle.com/datasets/alxmamaev/flowers-recognition
archive_url = 'https://drive.google.com/file/d/1PtGU9NM6hEHn2kEA7J4yVqI-1ci0iI2K/view?usp=sharing'
!mkdir dataset_flw
!gdown -O flowers.zip 1PtGU9NM6hEHn2kEA7J4yVqI-1ci0iI2K
!unzip flowers.zip -d ./dataset_flw

In [ ]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch import nn, optim
import numpy as np

# Путь до директории с изображениями
data_dir = './dataset_flw/flowers/'

# Запускаем на GPU, если есть
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Используется архитектура: {device}')

# Обработка картинок
dimension = 224 # Размер картинки до которого ресайзим все
transform = transforms.Compose([
    transforms.Resize((dimension, dimension)),  # Все картинки одного размера
    transforms.ToTensor(),         # Преобразуем картинку в тензор
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Нормализацяи изображерния
])

# Загружаем датасет из файлов в паках по классам
dataset = datasets.ImageFolder(data_dir, transform=transform)

# Получаем классы по названию папок
classes = dataset.classes

# Выводим инфо по количеству классов и картинок в них
class_counts = {}
print("Всего классов:", len(class_counts))
for img_path, label in dataset.imgs:
    class_name = classes[label]
    if class_name not in class_counts:
        class_counts[class_name] = 0
    class_counts[class_name] += 1
for class_name, count in class_counts.items():
    print(f'Класс "{class_name}" содержит {count} изображений')

# Разделяем данные на обучающую и тестовую выборки
train_test_ratio = 0.9
train_size = int(train_test_ratio * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Выводим количество изображений в обучающей и тестовой выборках
print(f'\nКоличество изображений: {len(dataset)}')
print(f'Обучающая выборка: {len(train_dataset)}')
print(f'Тестовая выборка: {len(test_dataset)}')

# Создаем загрузчики данных
batch_size = 10 # Количество картинок в батче
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Определем параметры для пулинг слоя
pooling_kernel_size = 4
pooling_stride = 2
pooling_padding = 2
pooling_count = 2

# Посчитаем размерность картинки после пулинг слоев
dimension_after_pooling = dimension
for i in range(pooling_count):
  dimension_after_pooling = int((dimension_after_pooling - pooling_kernel_size + 2 * pooling_padding) / pooling_stride + 1)
print (f'dimension_after_pooling: {dimension_after_pooling}')

# Определяем архитектуру сети
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Первый сверточный слой с выходными 16 каналами
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=1)
        # Добавляем батч-нормализацию к первому сверточному
        self.bn1 = nn.BatchNorm2d(num_features=16)
        # Пуллинг для уменьшения разрешения картинок
        self.pool = nn.MaxPool2d(kernel_size=4, stride=2, padding=2)
        # Второй  сверточный слой на 32 выходных канала
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        # Вторая батч-нормализация для второго сверточного слоя
        self.bn2 = nn.BatchNorm2d(num_features=32)
        # Первый линейный слой для преобразования в одномерный тензор на 64 нейрона
        self.fc1 = nn.Linear(32 * dimension_after_pooling * dimension_after_pooling, 64)
        # Батч-нормализация для одномерного тензора
        self.bn3 = nn.BatchNorm1d(num_features=64)
        # Второй линейный слой для раскладывания картинок по количеству классов
        self.fc2 = nn.Linear(64, len(classes))

    def forward(self, x):
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))  # свертка - нормализация - функция возбуждения - пулинг
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))  # свертка - нормализация - функция возбуждения - пулинг
        x = x.view(-1, 32 * dimension_after_pooling * dimension_after_pooling) # Преобразуем в одномерный тензор правильной размерности
        x = torch.relu(self.bn3(self.fc1(x)))               # линейный слой - нормализация - функция возбуждения
        x = self.fc2(x)                                     # второй линейный слой, выходной по классам
        return x

# Инициализируем модель в выбранной архитектуре
model = CNN().to(device)

# Инициализируем функцию потерь и оптимизатор Adam
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Обучаем модель за 5 циклов(эпох)
count_epoch = 5
for epoch in range(count_epoch):
    loss_batch = 0.0   # Потери по батчу
    correct = 0        # Кол-во правильно предсказанных
    total = 0          # Всего картинок

    for i, data in enumerate(train_loader, 0): # Идем по батчам в обучающей выборке
        # Распаковываем входные данные на отедльно картинки и классы
        inputs, labels = data
        # Загружаем данные в выбранную архитектуру
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()  # Сбрасываем градиенты

        outputs = model(inputs) # Прогоняем входные данные через модель
        loss = loss_func(outputs, labels) # Считаем потери
        # Вычисляем градиент функции потерь с обратным распространением
        loss.backward()
        optimizer.step()

        # Получаем предсказанный класс с максимальной вероятностью отнесения
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0) # Считаем сколько всего в батче файлов
        # Вычисляем кол-во корректно предсказанных картинок
        correct += (predicted == labels).sum().item()
        # Суммируем значение потерь по батчам
        loss_batch += loss.item()
    # Выводим
    print(f'Эпоха {epoch + 1} из {count_epoch}')
    print(f'Потери на обучающей выборке: {loss_batch / len(train_loader)}')
    print(f'Метрика успешности (Acc) модели: {(100 * correct / total):.2f}%\n')

# Проверяем модель на тестовой выборке
with torch.no_grad(): # Модель не обучаем, поэтому считаем без обратного распростронения
    correct = 0
    total = 0
    # Разбираем тестовый датасет на батчи
    for data in test_loader:
        images, labels = data
        # Загружаем данные  в выбранную архитектуру
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images) # Прогоняем мданные через модель
        # Получаем предсказанный класс с максимальной вероятностью отнесения
        _, predicted = torch.max(outputs.data, 1)
        # Считаем количество всех меток и количество корректно определенных
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Метрика успешности (Acc) модели на тестововй выборке: %.2f %%' % (100 * correct / total))
# Записваем модель в файл
torch.save(model.state_dict(), 'trained_model.pt')

Используется архитектура: cpu
Всего классов: 0
Класс "daisy" содержит 764 изображений
Класс "dandelion" содержит 1052 изображений
Класс "rose" содержит 784 изображений
Класс "sunflower" содержит 733 изображений
Класс "tulip" содержит 984 изображений

Количество изображений: 4317
Обучающая выборка: 3885
Тестовая выборка: 432
dimension_after_pooling: 57
Эпоха 1 из 5
Потери на обучающей выборке: 1.2085216596990747
Метрика успешности (Acc) модели: 49.73%

